In [1]:
# installing the library if needed
#!pip install psycopg2
#pip install petl

In [2]:
# HERE I WILL START THE PROJECT FROM YOUTUBE
import os
import petl
import psycopg2
import configparser  # working for simple configuration files
import requests      # http requests
import datetime
import json
import decimal

In [3]:
dbase_host = "localhost"
dbase_name = "test"
dbase_pass = "Postgresqlmahmoud7523"
dbase_user = "postgres"
dbase_port = "5432"

In [4]:
# defining connection object and connection string
try :
    conn = psycopg2.connect(
    database = dbase_name ,
    user = dbase_user ,
    password = dbase_pass,
    host = dbase_host ,
    port = dbase_port)
except Exception as e :
    print(e)

In [5]:
#so i dont have to commit the command every time
conn.set_session(autocommit=True)

In [6]:
# defining a cursor to execute a query
cur = conn.cursor()

In [7]:
# creating a table in the database with defined structure
expenses_create_query = """CREATE TABLE IF NOT EXISTS expenses  
(date TIMESTAMP,
rates decimal(6,5),
USD money,
CAD money)"""

try:
    cur.execute(expenses_create_query)
except Exception as e:
    print(e)


In [8]:
select_query = """ SELECT * FROM expenses"""

try:
    cur.execute(select_query) 
except psycopg2.Error as e: 
    print(e)

# FOR PRINTING RESULTS    
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(datetime.datetime(2020, 1, 11, 0, 0), Decimal('1.30510'), '$25.00', '$32.63')
(datetime.datetime(2020, 1, 30, 0, 0), Decimal('1.32170'), '$7.00', '$9.25')
(datetime.datetime(2020, 2, 11, 0, 0), Decimal('1.32920'), '$25.00', '$33.23')
(datetime.datetime(2020, 2, 28, 0, 0), Decimal('1.34290'), '$7.00', '$9.40')
(datetime.datetime(2020, 3, 11, 0, 0), Decimal('1.37450'), '$25.00', '$34.36')
(datetime.datetime(2020, 3, 30, 0, 0), Decimal('1.41560'), '$7.00', '$9.91')
(datetime.datetime(2020, 4, 11, 0, 0), Decimal('1.39850'), '$25.00', '$34.96')
(datetime.datetime(2020, 4, 30, 0, 0), Decimal('1.39100'), '$4.00', '$5.56')
(datetime.datetime(2020, 5, 11, 0, 0), Decimal('1.40120'), '$20.00', '$28.02')
(datetime.datetime(2020, 5, 30, 0, 0), Decimal('1.37870'), '$4.00', '$5.51')
(datetime.datetime(2020, 6, 11, 0, 0), Decimal('1.35530'), '$20.00', '$27.11')
(datetime.datetime(2020, 6, 17, 0, 0), Decimal('1.35550'), '$3.20', '$4.34')
(datetime.datetime(2020, 6, 30, 0, 0), Decimal('1.36280'), '$4.0

In [9]:
# reading data from a config file
config = configparser.ConfigParser()
try:
    config.read('configure.ini')
except Exception as e:
    print('could not read configuration file:' + str(e))
    #sys.exit()  #The sys.exit() function allows the developer to exit from Python  (i guess it is need when you are using a terminal)

In [10]:
# defing the variable from configuartion file to our scope notebook
startDate = config['CONFIG']['startDate']
url = config['CONFIG']['url']  # this url is missing the startdate
destServer = config['CONFIG']['server']        # this variable might be needed for some connection but not of my case
destDatabase = config['CONFIG']['database']   # this variable might be needed for some connection but not of my case

In [11]:
# getting data from the API
try:
    respond = requests.get(url+startDate)
except Exception as e:
    print("Could not make a request" + str(e))
    

In [12]:
# see how the result looks like
print(respond.text)

{
    "terms": {
        "url": "https://www.bankofcanada.ca/terms/"
    },
    "seriesDetail": {
        "FXUSDCAD": {
            "label": "USD/CAD",
            "description": "US dollar to Canadian dollar daily exchange rate",
            "dimension": {
                "key": "d",
                "name": "Date"
            }
        }
    },
    "observations": [
        {
            "d": "2020-01-02",
            "FXUSDCAD": {
                "v": "1.2992"
            }
        },
        {
            "d": "2020-01-03",
            "FXUSDCAD": {
                "v": "1.2988"
            }
        },
        {
            "d": "2020-01-06",
            "FXUSDCAD": {
                "v": "1.2970"
            }
        },
        {
            "d": "2020-01-07",
            "FXUSDCAD": {
                "v": "1.3009"
            }
        },
        {
            "d": "2020-01-08",
            "FXUSDCAD": {
                "v": "1.3026"
            }
        },
        {
          

In [13]:
# defing dates array to store dates in
dates_list = []

# defing rates array to store rates in
rates_list = []

# the request has a status respond code when it is 200 that means everything is OK
if (respond.status_code ==200):
    # defining a varable to parse the text as a json file
    bank_of_canandaRAW = json.loads(respond.text)
    
    # the data we consider is existed in an object called observations
    for row in bank_of_canandaRAW["observations"]:
        #striptime take a string a turns it into a date  , you also identify its structure or the format
        dates_list.append(datetime.datetime.strptime(row['d'],"%Y-%m-%d"))
        
        #decimal to turn the string into a decimal number
        rates_list.append(decimal.Decimal(row["FXUSDCAD"]['v']))
        

In [14]:
# loading the date and rates from api into a petl table
exchangeRates = petl.fromcolumns([dates_list,rates_list],header = ['date','rates'])

In [15]:
# printing results
print(exchangeRates)

+---------------------+--------+
| date                | rates  |
+=====================+========+
| 2020-01-02 00:00:00 | 1.2992 |
+---------------------+--------+
| 2020-01-03 00:00:00 | 1.2988 |
+---------------------+--------+
| 2020-01-06 00:00:00 | 1.2970 |
+---------------------+--------+
| 2020-01-07 00:00:00 | 1.3009 |
+---------------------+--------+
| 2020-01-08 00:00:00 | 1.3026 |
+---------------------+--------+
...



In [16]:
print(f"Number of rows is {exchangeRates.nrows()} rows")

Number of rows is 713 rows


In [17]:
# reading data from excel into a petl table
try:
    xlxs_expenses = petl.io.xlsx.fromxlsx("D:\Projects\Prpare for siParadigm\Expenses.xlsx")
except Exception as e:
    print(e)

In [18]:
print(xlxs_expenses)

+---------------------+-----+
| date                | USD |
+=====================+=====+
| 2020-01-11 00:00:00 |  25 |
+---------------------+-----+
| 2020-01-30 00:00:00 |   7 |
+---------------------+-----+
| 2020-02-11 00:00:00 |  25 |
+---------------------+-----+
| 2020-02-28 00:00:00 |   7 |
+---------------------+-----+
| 2020-03-11 00:00:00 |  25 |
+---------------------+-----+
...



In [19]:
print(f"Number of rows is {xlxs_expenses.nrows()} rows")

Number of rows is 26 rows


<h4>we have to make a full outer join so that to keep all of our expenses row <br></br>
In case we use inner join we might loose some row from expenses in case that there is no matching records in
exchange rates table</h4>

In [20]:
# the outer join return none when there is not a matching record
joined_table = petl.outerjoin(exchangeRates , xlxs_expenses , key = 'date')

In [21]:
joined_table.nrows()
#joined_table.lookall()
# NOte 719 - 712 = 7
# this 7 record are comming when the is no matching rows

720

In [22]:
# fill down missing values
# if you have a null value in a record it will fill it with the preceding one from its column 
# the second argument that is column you want to apply filldown function on (rates)
joined_table = petl.filldown(joined_table , 'rates')

In [23]:
#print(joined_table.lookall())

In [24]:
# Get rid of rows which are have None value in the usd columns
joined_table = petl.select(joined_table,lambda rec: rec.USD != None)
#joined_table.lookall()

In [25]:
joined_table.nrows()

26

In [26]:
# the target table has a column for expenses in CAD dollar and out table only has three
# Now we will add it to the tabel
joined_table = petl.addfield(joined_table ,"CAD" ,lambda rec : decimal.Decimal(rec.USD) * rec.rates)
joined_table

date,rates,USD,CAD
2020-01-11 00:00:00,1.3051,25,32.6275
2020-01-30 00:00:00,1.3217,7,9.2519
2020-02-11 00:00:00,1.3292,25,33.2300
2020-02-28 00:00:00,1.3429,7,9.4003
2020-03-11 00:00:00,1.3745,25,34.3625


In [27]:
# to be matching with database field names
joined_table = petl.rename(joined_table,{ 'USD': 'usd' , 'CAD': 'cad' })
#joined_table = petl.head(joined_table , 5)

In [28]:
# population of out table into database
try :
    petl.io.todb(joined_table, conn, 'expenses') 
except Exception as e:
    print(e)

In [ ]:
# in case you want to see the inserted data uncomment the next code  
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

In [29]:
# for the sake of this demo i will drop the table
try:
    cur.execute("DROP table expenses")
except Exception as e:
    print(e)

In [30]:
cur.close()
conn.close()